In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Evaluation

In order to pick the best move instead of just a random one, our engine needs some way to evaluate chess positions. To this end, we will define a basic interface `Evaluator` and a concrete implementation `PieceEvaluator` that will be capable of two things:
* Evaluate terminal state positions
* Evaluate all other position by using a heuristic

As the code will be more complex than before, classes will not be shown as one block of code, but rather methods and attributes will be added dynamically one after another. This form of dynamic class modification is called [monkey patching](https://docs.plone.org/appendices/glossary.html#term-Monkey-patch) in the python community and is often used as a technique to patch third party code.

To evaluate terminal states, we first need to define what a terminal state is. These are all the end positions of finished games. 
A finished game is either won by one of the two players or a draw. To check whether the game is won, the method `is_checkmate()` of the `chess.Board` class can be used. 
For a draw there are multiple functions for various conditions like `is_stalemate` or `is_insufficient_material`. 
To make the code more expressive a new function `is_draw` is added to the `chess.Board` class that checks for any of these conditions.

In [ ]:
import chess


def is_draw(self) -> bool:
    return self.is_stalemate() or self.is_insufficient_material(
    ) or self.is_fivefold_repetition() or self.is_seventyfive_moves()


chess.Board.is_draw = is_draw

These two methods allows us to implement the basic interface `Evaluator`. The method `evaluate` takes only the board `board` as a parameter and returns the score of the game if it is finished and `None` otherwise. The score used the class `PovScore` from the `python-chess` library, which is a `chess.engine.Score` combined with the point of view. The second method is `evaluate` which has the same signature as `evaluate` and evaluates unfinished game positions. This method will be provided by concrete subclasses.

In [ ]:
from chess.engine import Score, PovScore


class Evaluator():

    def evaluate(self, board: chess.Board) -> PovScore:
        if board.is_checkmate():
            return PovScore(Mate(0), board.turn)
        if board.is_draw():
            return PovScore(Cp(0), board.turn)
        return None

    def heuristic(self, board: chess.Board) -> PovScore:
        pass

Next, a concrete subclass `PieceEvaluator` is implemented that provides the `heuristic`. This simple heuristic will calculate the value of the position based on the material on the board. The class inherits from the previously defined class `Evaluator`. The constructor takes a parameter `piece_values`, which is a dict that assigns each piece type a value.

In [ ]:
from chess.engine import Score


class PieceEvaluator(Evaluator):

    def __init__(self, piece_values: dict):
        self.piece_values = piece_values

We can then provide the implementation for the `heuristic` method. The evaluation of an unfinished game is determined by summing up the piece values of the player to move
and subtracting the piece values of the opponent. A positive score therefore is good for player to move whereas a negative score is good for the opponent. To efficiently iterate over all squares with pieces, the board representation `board.occupied_co` can be used together with the `scan_reversed` function of python-chess. The expression `chess.scan_reversed(board.occupied_co[board.turn])` thus yields each square that contains a piece of the player to move.

In [ ]:
from chess.engine import Cp, Score, PovScore


def heuristic(self, board: chess.Board) -> PovScore:
    player_score = 0
    for square in chess.scan_reversed(board.occupied_co[board.turn]):
        player_score += self.piece_values[board.piece_type_at(square)]

    opponent_score = 0
    for square in chess.scan_reversed(board.occupied_co[not board.turn]):
        opponent_score += self.piece_values[board.piece_type_at(square)]

    return PovScore(Cp(player_score - opponent_score), board.turn)


PieceEvaluator.heuristic = heuristic

To actually create an instance of the `PieceEvaluator` we need to define scores for each piece type. A very common [piece value assignment](https://en.wikipedia.org/wiki/Chess_piece_relative_value) is used for this.

In [ ]:
STANDARD_PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}

Then the instance can be created with these values.

In [ ]:
standard_evaluator = PieceEvaluator(piece_values=STANDARD_PIECE_VALUES)

We can write some tests for the evaluator to make sure it is working correctly. The first test checks whether `evaluate` correctly detects a mate.

In [ ]:
import IPython.display
from chess.engine import Mate, PovScore
import chess

SCHOLAR_MATE = "r1bqkb1r/pppp1Qpp/2n2n2/4p3/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 0 4"
board = chess.Board(SCHOLAR_MATE)
IPython.display.display(board)

# It is blacks turn and he was checkmated.
score = standard_evaluator.evaluate(board)
assert score == PovScore(Mate(0), chess.BLACK), f"{score} != {PovScore(Mate(0), chess.BLACK)}"

The second case will be tested with a stalemate position. In this case the `evaluate` method should return the score for a draw.

In [ ]:
from chess.engine import Cp, PovScore
import chess

# Viktor Korchnoi vs Anatoly Karpov, World Champtionchip 5th game 1987
STALE_MATE = "8/5KBk/8/8/p7/P7/8/8 b - - 34 124"
board = chess.Board(STALE_MATE)
IPython.display.display(board)

# It is a stalemate, so the score should be zero
score = standard_evaluator.evaluate(board)
assert score == PovScore(Cp(0), chess.BLACK), f"{score} != {PovScore(Cp(0), chess.BLACK)}"

The last case to be tested is a game that is not finished. The `evaluate` method should return `None` and instead `heuristic` should calculate the score.

In [ ]:
from chess.engine import Cp, PovScore
import chess

# Topalov, Veselin (2740) vs. Shirov, Alexei (2710)
SHIROV_SACRIFICE = "8/8/4kpp1/3p1b2/p6P/2B5/6P1/6K1 b - - 0 47"
board = chess.Board(SHIROV_SACRIFICE)
IPython.display.display(board)

# Game not finished, so score should be None
score = standard_evaluator.evaluate(board)
assert score == None, f"{score} != None"

# Black has 2 points more and is about to move, so the score should be 2
score = standard_evaluator.heuristic(board)
assert score == PovScore(Cp(2), chess.BLACK), f"{score} != 2"

To make sure the `heursitic` method is working correctly, it will be tested on another game.

In [ ]:
from chess.engine import Cp, PovScore
import chess

# Evgeny Yuryevich Vladimirov vs. Vladimir Viktorovich Epishin
VLADIMIROV_THUNDERBOLT = "r4k1r/1b2bPR1/p4n2/3p4/4P2P/1q2B2B/PpP5/1K4R1 w - - 0 26"
board = chess.Board(VLADIMIROV_THUNDERBOLT)
IPython.display.display(board)

# Game not finished, so score should be None
score = standard_evaluator.evaluate(board)
assert score == None, f"{score} != None"

# Black has 10 points more, but White is about to move, so the score should be -10
score = standard_evaluator.heuristic(board)
assert score == PovScore(Cp(-10), chess.WHITE), f"{score} != -10"

To use this newly created Evaluator a new Engine `EvaluationEngine` is created that inherits from the `Engine` class. It expects an instance `evaluator` of type `Evaluator` to be passed as an argument.

In [ ]:
import random
import chess.engine
from converted_notebooks.s04_engine_interface import Engine, ScoredMove


class EvaluationEngine(Engine):

    def __init__(self, evaluator: Evaluator):
        self.evaluator = evaluator

To get the value of a possible move, 
we define a method `_evaluate_move` 
which takes d a chess board `board`,
and a chess move `move`
as parameters
and returns a `ScoredMove`.

It uses the evaluator to determine the score of the board after the move have been made. Additionally, the score is turned into an absolute value by using the `white` method of the `PovScore` class

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move) -> ScoredMove:
    board.push(move)
    score = self.evaluator.evaluate(board)
    if score is None:
        score = self.evaluator.heuristic(board)
    board.pop()

    return ScoredMove(score.white(), move)


EvaluationEngine._evaluate_move = _evaluate_move

To get the value of all legal moves,
we define the method `_evaluate_moves`
which takes a chess board `board` as a parameter.
It returns a list of `ScoredMove` objects by evaluating all legal moves.

In [ ]:
def _evaluate_moves(self, board: chess.Board) -> list[ScoredMove]:
    return [self._evaluate_move(board, move) for move in board.legal_moves]


EvaluationEngine._evaluate_moves = _evaluate_moves

Next, the `analyse` method can be defined using the `_evaluate_moves` method.

The scored moves are then shuffled and, afterwards, sorted by their score, 
creating a different order of moves having the same score, 
depending on the state of the RNG. By default, the python `sort` method sorts from lowest to highest. 
Therefore, the first move is the best for Black, unless the sorting order is reversed.

In [ ]:
def analyse(self, board: chess.Board) -> list[ScoredMove]:
    nextMoves = self._evaluate_moves(board.copy())
    random.shuffle(nextMoves)

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves


EvaluationEngine.analyse = analyse

At last the `play` method needs to be implemented.
This is done by taking the best scored move as determined by the `analyse` method 
and returning it as a `chess.engine.PlayResult`.


In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    bestScoredMove = self.analyse(board)[0]
    return chess.engine.PlayResult(move=bestScoredMove.move, ponder=None)


EvaluationEngine.play = play

As before we can let the new defined engine play against the `RandomEngine`. Through the use of the evaluation function it should already play significantly better and win most of the games.

In [ ]:
import random
from converted_notebooks.s06_play import play_game
from converted_notebooks.s05_random_engine import RandomEngine

random.seed(42)

board = chess.Board()
play_game(board, EvaluationEngine(evaluator=standard_evaluator), RandomEngine())
print(board.outcome())